# Data loading

In [2]:
BASIC_PATH = '../Data/'
ALL_FILES = BASIC_PATH + '*.csv'

import pandas as pd
import numpy as np
import glob as glob

def readAllFiles():
    files = glob.glob(ALL_FILES)
    frames = []

    for file in files:
        df = pd.read_csv(file, index_col = 0)
        frames.append(df)

    return pd.concat(frames)

df = readAllFiles()

# Predict arrival delay with linear regression

In [15]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
enc = LabelEncoder();

X = df.loc[:,['date', 'airline', 'departure_airport', 'arrival_airport']]
X.date = enc.fit_transform(X.date)
X.airline = enc.fit_transform(X.airline)
X.departure_airport = enc.fit_transform(X.departure_airport)
X.arrival_airport = enc.fit_transform(X.arrival_airport)

y = df.loc[:,['arrival_delay']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

model = LinearRegression()
model.fit(X_train, y_train)

y_predict = model.predict(X_test)

# The coefficients
print('Coefficients: \n', model.coef_)

# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_predict))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.5f' % r2_score(y_test, y_predict))

Coefficients: 
 [[ 0.00203808 -0.10865095  0.01613986]]
Mean squared error: 1328.77
Variance score: 0.00086


# Predict arrival delay with Random Forest Regressor and use onehotencoder

In [5]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder 

test = df

labelEnc = LabelEncoder();
onehotEnc = OneHotEncoder(sparse=False)

integer_encoded = labelEnc.fit_transform(test.airline)
zipped = zip(integer_encoded, test.date)
label_airports = list(set(list(zipped)))
label_airports.sort(key = lambda x:x[0])

integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehotEnc.fit_transform(integer_encoded)
b = np.array(test[['airline_code','arrival_lon', 'arrival_lat','departure_lat', 'daparture_lon']])
X = np.hstack((onehot_encoded, b))

y = test.loc[:,['arrival_delay']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

model = RandomForestRegressor()
model.fit(X_train, y_train.values.ravel())

y_predict = model.predict(X_test)

# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_predict))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.5f' % r2_score(y_test, y_predict))

KeyError: "['daparture_lon'] not in index"

# Predict arrival delay with Random Forest Regressor

In [7]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
enc = LabelEncoder();

X = df.loc[:,['date', 'airline', 'departure_airport', 'arrival_airport']]
X.date = enc.fit_transform(X.date)
X.airline = enc.fit_transform(X.airline)
X.departure_airport = enc.fit_transform(X.departure_airport)
X.arrival_airport = enc.fit_transform(X.arrival_airport)

y = df.loc[:,['arrival_delay']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

model = RandomForestRegressor()
model.fit(X_train, y_train.values.ravel())

y_predict = model.predict(X_test)

# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_predict))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.5f' % r2_score(y_test, y_predict))

Mean squared error: 1443.57
Variance score: -0.08546


# Predict arrival delay with Ridge Regression

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge

enc = LabelEncoder();

X = df.loc[:,['date', 'airline', 'departure_airport', 'arrival_airport']]
X.date = enc.fit_transform(X.date)
X.airline = enc.fit_transform(X.airline)
X.departure_airport = enc.fit_transform(X.departure_airport)
X.arrival_airport = enc.fit_transform(X.arrival_airport)

y = df.loc[:,['arrival_delay']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

model = Ridge(alpha=0.3,normalize=True)
model.fit(X_train, y_train.values.ravel())

y_predict = model.predict(X_test)

# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_predict))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.5f' % r2_score(y_test, y_predict))

# Classificatie test

In [12]:
from sklearn import datasets
from sklearn import cross_validation
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
enc = LabelEncoder();

X = df.loc[:,['date', 'airline', 'departure_airport', 'arrival_airport']]
X.date = enc.fit_transform(X.date)
X.airline = enc.fit_transform(X.airline)
X.departure_airport = enc.fit_transform(X.departure_airport)
X.arrival_airport = enc.fit_transform(X.arrival_airport)

y = df.loc[:,['arrival_delay']]

X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), random_state=0)
    
clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(y_pred)
print(y_test)

accuracy = metrics.accuracy_score(y_test, y_pred)

print(accuracy)


ValueError: Unknown label type: (array([ -139.,  -115.,   -94., ...,  1623.,  1666.,  1823.]),)